In [20]:
import requests
import json

url = "https://www.ixigo.com/outlook/v1/onward/ranged?departureDate=03092025&destination=BOM&fareClass=e&origin=PNQ&paxCombinationType=100&refundTypes=REFUNDABLE%2CNON_REFUNDABLE%2CPARTIALLY_REFUNDABLE"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept": "application/json"
}

resp = requests.get(url, headers=headers)

if resp.status_code == 200:
    data = resp.json()
    print("Top-level keys:", list(data.keys()))
else:
    print("Failed:", resp.status_code, resp.text[:500])


Top-level keys: ['data', 'errors']


In [21]:
import requests
import json

url = "https://www.ixigo.com/outlook/v1/onward/ranged?departureDate=03092025&destination=BOM&fareClass=e&origin=PNQ&paxCombinationType=100&refundTypes=REFUNDABLE%2CNON_REFUNDABLE%2CPARTIALLY_REFUNDABLE"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept": "application/json"
}

resp = requests.get(url, headers=headers)

if resp.status_code == 200:
    data = resp.json()

    # Save full response for inspection
    with open("raw.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

    # Show keys inside data
    if "data" in data:
        print("Keys under data:", list(data["data"].keys()))
    else:
        print("No 'data' key found")

    # Show any errors
    if "errors" in data and data["errors"]:
        print("Errors:", data["errors"])
else:
    print("Failed:", resp.status_code, resp.text[:500])


Keys under data: ['returning', 'going', 'showOutlook']


In [22]:
import requests
import json

url = "https://www.ixigo.com/outlook/v1/onward/ranged?departureDate=03092025&destination=BOM&fareClass=e&origin=PNQ&paxCombinationType=100&refundTypes=REFUNDABLE%2CNON_REFUNDABLE%2CPARTIALLY_REFUNDABLE"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept": "application/json"
}

resp = requests.get(url, headers=headers)

if resp.status_code == 200:
    data = resp.json()

    flights = []
    for trip in data.get("data", {}).get("going", []):
        details = trip.get("flightDetails", [])
        fares = trip.get("fares", [])
        
        if details:
            seg = details[0]  # first segment
            flight = {
                "airline": seg.get("airlineName"),
                "flightNo": seg.get("flightNumber"),
                "depart": seg.get("departDateTimeDisplay"),
                "arrive": seg.get("arrivalDateTimeDisplay"),
                "duration": seg.get("duration", {}).get("text"),  # e.g. "1h 10m"
                "price": fares[0].get("fareDetails", {}).get("displayFare") if fares else None
            }
            flights.append(flight)

    # Save to file
    with open("flights.json", "w", encoding="utf-8") as f:
        json.dump(flights, f, indent=2)

    if flights:
        print("Flights saved in flights.json")
        for f in flights[:5]:  # show first 5
            print(f)
    else:
        print("No flights found in 'going'")
else:
    print("Request failed:", resp.status_code, resp.text[:500])


AttributeError: 'str' object has no attribute 'get'

In [16]:
df.head()

,airline,airlineCode,flightNumber,date,found,fare,providerId,searchId
0,NaN,NaN,NaN,02-09-2025,1756749159000,8388.0,1044,01092025232239000$1044
1,NaN,NaN,NaN,03-09-2025,1756835277000,8189.0,1044,02092025231757000$1044
2,NaN,NaN,NaN,04-09-2025,1756834860000,7129.0,1044,02092025231100000$1044
3,NaN,NaN,NaN,05-09-2025,1756833423000,7129.0,1044,02092025224703000$1044
4,NaN,NaN,NaN,06-09-2025,1756824703000,7129.0,1044,02092025202143000$1044


In [24]:
for i, trip in enumerate(data.get("data", {}).get("going", [])):
    print(i, type(trip))
    if isinstance(trip, dict):
        print("Keys:", trip.keys())
    else:
        print("Value:", trip)
    if i > 5:
        break


0 <class 'str'>
Value: origin
1 <class 'str'>
Value: destination
2 <class 'str'>
Value: startDate
3 <class 'str'>
Value: endDate
4 <class 'str'>
Value: results
5 <class 'str'>
Value: ranges


In [25]:
import json
import csv

# Load your JSON file
with open("ixigo_response.json", "r") as f:
    data = json.load(f)

# Prepare list for storing flight info
flights = []

# Check that 'data' and 'going' exist
going_data = data.get("data", {}).get("going", {})
results = going_data.get("results", [])

# Iterate over flight results
for trip in results:
    # Each trip should have 'flightDetails' and 'fares'
    flight_details = trip.get("flightDetails", [])
    fares = trip.get("fares", [])

    if not flight_details:
        continue

    # Extract relevant info from each segment
    for idx, fd in enumerate(flight_details):
        fare = fares[idx] if idx < len(fares) else {}
        duration = fd.get("duration", {}).get("timeInHours", "N/A")
        airline = ", ".join(fd.get("airline", []))
        origin = fd.get("origin", "N/A")
        destination = fd.get("destination", "N/A")
        stops = fd.get("stops", "N/A")
        price = fare.get("displayFare", "N/A")

        flights.append({
            "Origin": origin,
            "Destination": destination,
            "Airline": airline,
            "Stops": stops,
            "Duration (hrs)": duration,
            "Price": price
        })

# Save to CSV
csv_file = "ixigo_flights.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=flights[0].keys())
    writer.writeheader()
    writer.writerows(flights)

print(f"Saved {len(flights)} flights to {csv_file} ✅")


FileNotFoundError: [Errno 2] No such file or directory: 'ixigo_response.json'